## Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |

Suponha que temos uma tabela de nutrientes de diferentes tipos de alimentos. Sabendo o valor
diário de referência (VDR) de cada nutriente e o preço de cada unidade de alimento, qual é a
dieta ótima que contém pelo menos o valor diário de referência e seja de menor custo?


# 1)

In [1]:
!pip install gurobipy

In [2]:
from gurobipy import Model, GRB, quicksum

# Criar o modelo
modelo = Model("McDieta")

Restricted license - for non-production use only - expires 2026-11-23


## Declaração das variáveis

In [4]:
from google.colab import files
import pandas as pd

alimentos = []
custos = []

# Fazer o upload do arquivo
uploaded = files.upload()

# Obter o nome do arquivo carregado
nutrientes_file_name = list(uploaded.keys())[0]
custos_file_name = list(uploaded.keys())[1]
vdr_file_name = list(uploaded.keys())[2]

# Ler o arquivo com pandas (substitua 'sep' pelo separador correto)
nutrientes_file = pd.read_csv(nutrientes_file_name, sep='\s+', header=1, index_col=0)
custos_file = pd.read_csv(custos_file_name, sep='\s+', header=2, index_col=0)
vdr_file = pd.read_csv(vdr_file_name, sep='\s+', header=1, index_col=0)

df_nutrientes = pd.DataFrame(nutrientes_file)

# Criando uma variável para representar a quantidade de cada alimento:
for i in range(len(df_nutrientes.index)):
  # Descomentar se quiser utilizar o UpperBound para responder a questão 4:
  # alimentos.append(modelo.addVar(name=df_nutrientes.index[i], vtype=GRB.CONTINUOUS, lb=0, ub=1))
  alimentos.append(modelo.addVar(name=df_nutrientes.index[i], vtype=GRB.CONTINUOUS, lb=0))

# Matriz contendo os nutrientes de cada alimento:
valores_nutrientes = df_nutrientes.to_numpy()

# Armazenando os custos de cada alimento:
df_custos = pd.DataFrame(custos_file)
valores_custos = df_custos.to_numpy()

# Percorrendo a matriz e pegando apenas a informação dos custos e salvando no vetor
for i in range(len(valores_custos)):
    custos.append(valores_custos[i][0])

nutrientes_valores = valores_nutrientes.transpose()


Saving McDonalds-amnt.wsv to McDonalds-amnt (9).wsv
Saving McDonalds-food.wsv to McDonalds-food (9).wsv
Saving McDonalds-nutr.wsv to McDonalds-nutr (9).wsv
[6.9, 9.4, 6.4, 7.9, 6.9, 6.9, 6.4, 6.9, 6.9, 3.5, 3.15, 2.65, 2.5, 3.5, 3.5, 10.45, 13.75, 13.75, 1.5, 1.5, 1.5, 4.45, 3.5, 2.65, 8.45, 3.5, 5.25, 1.0, 1.0, 1.0, 1.0, 2.7, 3.2, 2.7, 3.2, 2.7, 3.2, 2.7, 3.2, 2.9, 2.5, 1.5, 1.5, 2.5, 2.9, 3.2, 2.5, 2.9, 3.2, 2.5, 2.9, 3.2, 2.5, 2.9, 3.2, 2.5, 2.9, 3.2, 1.5, 1.5, 1.5, 2.0, 2.0, 2.45, 2.45, 3.5, 3.5, 3.5, 3.95, 3.95, 3.95, 3.95, 3.95, 3.95, 4.95, 3.95, 4.95, 3.95, 4.95, 2.25, 2.25, 3.6, 1.0, 2.5]


## Função Objetivo

In [5]:
funcao_objetivo = quicksum(custos[i] * alimentos[i] for i in range(len(alimentos)))

modelo.setObjective(funcao_objetivo, GRB.MINIMIZE)

## Restrições

In [6]:
for nutriente_indice in range(len(nutrientes_valores)):
  total_nutriente = 0
  for alimento in range(len(nutrientes_valores[nutriente_indice])):
    total_nutriente += nutrientes_valores[nutriente_indice][alimento] * alimentos[alimento]
  modelo.addConstr(total_nutriente >= 100, "O VDR de cada nutriente é 100")

## Resolvendo o problema

In [7]:
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 84 columns and 634 nonzeros
Model fingerprint: 0x8971e040
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 16 columns
Presolve time: 0.01s
Presolved: 10 rows, 68 columns, 580 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+01   0.000000e+00      0s
       3    2.4311927e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.431192661e+01


## Imprimindo a solução do problema

In [8]:
if(modelo.status == GRB.OPTIMAL):
  print(f"Valor Ótimo: {modelo.objVal}")

Valor Ótimo: 24.31192660550459


## Resposta: R$ 24,31

# 2)

## Análise de Sensibilidade

In [9]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in modelo.getVars():
  print(f"Variável {variavel.VarName}:")
  print(f"  lower bound do custo (SAObjLow): {variavel.SAObjLow}")
  print(f"  upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in modelo.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  upper bound do recurso (SARHSUp): {restricao.SARHSUp}")


Intervalos de Custos (Vetor de Custos):
Variável Big Mac:
  lower bound do custo (SAObjLow): 3.8954128440366977
  upper bound do custo (SAObjUp): inf
Variável Big Tasty:
  lower bound do custo (SAObjLow): 6.279816513761467
  upper bound do custo (SAObjUp): inf
Variável Quarterão com Queijo:
  lower bound do custo (SAObjLow): 5.077064220183487
  upper bound do custo (SAObjUp): inf
Variável McNífico Bacon:
  lower bound do custo (SAObjLow): 5.4779816513761475
  upper bound do custo (SAObjUp): inf
Variável Cheddar McMelt:
  lower bound do custo (SAObjLow): 4.676146788990826
  upper bound do custo (SAObjUp): inf
Variável McFish:
  lower bound do custo (SAObjLow): 3.144036697247707
  upper bound do custo (SAObjUp): inf
Variável McChicken:
  lower bound do custo (SAObjLow): 3.441284403669725
  upper bound do custo (SAObjUp): inf
Variável Crispy Chicken:
  lower bound do custo (SAObjLow): 3.7724770642201837
  upper bound do custo (SAObjUp): inf
Variável Chicken Grill:
  lower bound do custo 

**1.** Variáveis de Decisão (alimentos): Os intervalos de sensibilidade para o custo (SAObjLow e SAObjUp) mostram os limites dentro dos quais o coeficiente objetivo de cada variável pode variar sem alterar a base ótima da solução. Isso significa que:

Para a maioria dos alimentos, o limite inferior (SAObjLow) indica o menor valor de custo que ainda manterá a solução ótima inalterada.
Muitos alimentos têm o limite superior (SAObjUp) como infinito, o que significa que seus custos podem aumentar indefinidamente sem impactar a base ótima atual.
Por exemplo, o "Big Mac" tem um limite inferior de 3.89, o que implica que, se o custo cair abaixo desse valor, a solução ótima pode mudar.

**2.** Restrições (Recursos): Os intervalos de sensibilidade para os recursos (SARHSLow e SARHSUp) indicam as variações permitidas nos limites das restrições (como VDR de nutrientes) sem alterar a viabilidade da solução:

SARHSLow: Valor mínimo que a restrição pode assumir sem afetar a solução ótima.

SARHSUp: Valor máximo permitido antes que a solução deixe de ser ótima.
Por exemplo, na restrição com limite superior de 136.69, se o valor aumentar além disso, a solução ótima não será mais válida.

Portanto, A análise mostra que há flexibilidade para alterações nos custos de alguns alimentos e nos limites de restrições. Contudo, certos alimentos (como "McDuplo" e "Casquinha Baunilha") possuem intervalos mais estreitos, indicando maior sensibilidade a mudanças nos custos. Restrições com limites inferiores positivos também destacam nutrientes que podem afetar a viabilidade da solução com pequenas variações.








# 3)

## Declaração das variáveis

In [4]:
alimentos_veg = []
alimentos_veg_indices = []
custos_veg = []

# Percorrendo a matriz e pegando apenas a informação dos custos e salvando no vetor
for i in range(len(valores_custos)):
    if ( valores_custos[i][3] ):
      alimentos_veg_indices.append(i)
      custos_veg.append(valores_custos[i][0])

# Criando uma variável para representar a quantidade de cada alimento vegetariano:
for i in range(len(df_nutrientes.index)):
  if ( i in alimentos_veg_indices ):
    alimentos_veg.append(modelo.addVar(name=df_nutrientes.index[i], vtype=GRB.CONTINUOUS, lb=0))


## Função Objetivo

In [5]:
funcao_objetivo = quicksum(custos_veg[i] * alimentos_veg[i] for i in range(len(alimentos_veg)))

modelo.setObjective(funcao_objetivo, GRB.MINIMIZE)

## Restrições

In [6]:
for nutriente_indice in range(len(nutrientes_valores)):
  total_nutriente = 0
  for alimento in range(len(nutrientes_valores[nutriente_indice])):
    if ( alimento in alimentos_veg_indices ):
      indice_correspondente = alimentos_veg_indices.index(alimento)
      total_nutriente += nutrientes_valores[nutriente_indice][alimento] * alimentos_veg[indice_correspondente]
  modelo.addConstr(total_nutriente >= 100, "O VDR de cada nutriente é 100")

## Resolvendo o problema dos vegetarianos

In [7]:
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 151 columns and 464 nonzeros
Model fingerprint: 0x921708cf
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 99 columns
Presolve time: 0.02s
Presolved: 10 rows, 52 columns, 415 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.750000e+01   0.000000e+00      0s
       4    3.9634146e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds (0.00 work units)
Optimal objective  3.963414634e+01


## Imprimindo a Solução

In [8]:
if(modelo.status == GRB.OPTIMAL):
  print(f"Valor Ótimo: {modelo.objVal}")

Valor Ótimo: 39.63414634146341


## Resposta: R$ 39,63

# 4)

## Para fazer essa questão, basta adicionar na declaração das variáveis de decisão um limite superior igual a 1. Dessa forma, limita-se a quantidade de alimento para 1. Depois é só rodar novamente cada parte de código. E imprimir a solução ótima

In [ ]:
if(modelo.status == GRB.OPTIMAL):
  print(f"Valor Ótimo: {modelo.objVal}")

Valor Ótimo: 31.562784549860485


## Resposta: R$ 31,56